In [1]:
# COLMAP
%cd /notebooks
!pip install -q plyfile pycolmap Pillow==9.5.0 pycolmap plotly

!sudo apt-get update
!sudo apt-get install libglm-dev

!pip install -q /notebooks/diff-gaussian-rasterization
!pip install -q /notebooks/gaussian-splatting/submodules/simple-knn

!apt install imagemagick -y
# remove line 88 of vim ~/../etc/ImageMagick-6/policy.xml to run MoviePy
!sed -i '88d' ~/../etc/ImageMagick-6/policy.xml 

!sudo apt-get update && sudo apt-get install -y \
	ninja-build \
	libeigen3-dev \
	libflann-dev \
	libsqlite3-dev \
	libgl1-mesa-dev \
	cmake \
	libgoogle-glog-dev \
	libgflags-dev \
	libsuitesparse-dev \
	libceres-dev \
	libcgal-dev \
	libcgal-qt5-dev \
	libmetis-dev \
	libfreeimage-dev \
	libglew-dev

!sudo updatedb   # update the database used by locate
!locate Eigen3Config.cmake
!locate libflann

# # # Build COLMAP
# !git clone https://github.com/colmap/colmap.git
# %cd /notebooks/colmap
# !rm -rf build
# !mkdir build
# %cd /notebooks/colmap/build
# !cmake .. -DCMAKE_CUDA_ARCHITECTURES=native -GNinja

%cd /notebooks/colmap/build
!ninja
!sudo ninja install

# Check if colmap is installed
!colmap -h

%cd /notebooks


#HLOC
!pip install mediapy

# !rm -rf /notebooks/Hierarchical-Localization
# !git clone --quiet --recursive https://github.com/cvg/Hierarchical-Localization/

%cd /notebooks/Hierarchical-Localization
!pip install --progress-bar off --quiet -e .


%cd /notebooks

/notebooks
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:3 https://deb.nodesource.com/node_16.x focal InRelease [4583 B]            
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1188 kB]
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:7 https://deb.nodesource.com/node_16.x focal/main amd64 Packages [776 B]   
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2896 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1117 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [29.3 kB]
Get:12 http://security.ubuntu.com/u

In [1]:
# Change to the appropriate directory
%cd /notebooks/Hierarchical-Localization

# Import libraries from hloc
from hloc import (
	extract_features, 
	match_features, 
	reconstruction, 
	visualization, 
	pairs_from_retrieval, 
	colmap_from_nvm, 
	pairs_from_exhaustive, 
	pairs_from_covisibility
)
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

# Update system path
import sys
third_party_path = '/notebooks/Hierarchical-Localization/third_party'
sys.path.extend([
	third_party_path, 
	f'{third_party_path}/d2net',
	f'{third_party_path}/deep-image-retrieval',
	f'{third_party_path}/r2d2',
	f'{third_party_path}/SuperGluePretrainedNetwork'
])

# Navigate back to the notebooks directory
%cd /notebooks

# Import standard libraries
import os
import io
import glob
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime, timedelta
from PIL import Image, ImageFilter, ImageChops

# Import additional libraries
import mediapy as media
import torch
import pycolmap
import sqlite3

/notebooks/Hierarchical-Localization
/notebooks


In [7]:
# Connect to the database
%cd /notebooks
connection = sqlite3.connect('/notebooks/dbs/db_session_1535AC7C-7030-431C-8E1F-AF0DBEF80A9B.sqlite')

# # Define the search parameters
# tl_lat = 160.7128
# tl_long = -74.0060
# br_lat = 10.6977
# br_long = -13.9792

# cursor = connection.cursor()
# cursor.execute("SELECT image_blob FROM maptodon_raw WHERE lat BETWEEN ? AND ? AND long BETWEEN ? AND ?", (br_lat, tl_lat, tl_long, br_long))
# results = cursor.fetchall()

# Get all data
cursor = connection.cursor()
cursor.execute("SELECT image_blob FROM maptodon_raw")
results = cursor.fetchall()

# Delete working image, and create a new one
!rm -rf working
!mkdir working
!mkdir working/input

# Process the search results
image_hashes = {}
if len(results) > 0:
	for i, row in enumerate(results):
		image_data = row[0]

		# Save to working directory
		image = Image.open(io.BytesIO(image_data))

		image.save("./working/input/image_{:05d}.png".format(i))
	print("Saved " + str(len(results)) + " images to working directory.")
else:
	print("No results found.")

# Close the database connection
connection.close()

/notebooks
Saved 826 images to working directory.


In [8]:
# Delte to 1/3 images.
images = glob.glob("./working/input/*.png")
for i, image in enumerate(images):
	if i % 3 != 0:
		os.remove(image)
images = glob.glob("./working/input/*.png")
print(f"There are {len(images)} images in the working directory.")

There are 276 images in the working directory.


In [9]:
%%time
%cd /notebooks

location = Path('./working')
images = location / 'input'
image_list = [path.split(str(images.stem) + "/")[-1] for path in sorted(glob.glob(str(images / '*')))]
outputs = location / 'sfm'
sfm_pairs = outputs / 'pairs-sfm.txt'
loc_pairs = outputs / 'pairs-loc.txt'
sfm_dir = outputs / 'sfm'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'

feature_conf = extract_features.confs['superpoint_max']
matcher_conf = match_features.confs['superglue']

extract_features.main(feature_conf, images, image_list=image_list, feature_path=features)
pairs_from_exhaustive.main(sfm_pairs, image_list=image_list)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)

model = reconstruction.main(sfm_dir, images, sfm_pairs, features, matches, image_list=image_list)

fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping", points_rgb=True)
fig.show()

[2023/10/15 04:26:10 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-rmax1600',
 'preprocessing': {'grayscale': True, 'resize_force': True, 'resize_max': 1600}}


/notebooks
Loaded SuperPoint model


100%|██████████| 276/276 [00:09<00:00, 28.13it/s]
[2023/10/15 04:26:21 hloc INFO] Finished exporting features.
[2023/10/15 04:26:21 hloc INFO] Found 37950 pairs.
[2023/10/15 04:26:21 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}


Loaded SuperGlue model ("outdoor" weights)


100%|██████████| 37950/37950 [27:23<00:00, 23.09it/s]
[2023/10/15 04:53:45 hloc INFO] Finished exporting matches.
[2023/10/15 04:53:45 hloc INFO] Creating an empty database...
[2023/10/15 04:53:45 hloc INFO] Importing images into the database...
[2023/10/15 04:53:52 hloc INFO] Importing features into the database...
100%|██████████| 276/276 [00:00<00:00, 856.46it/s]
[2023/10/15 04:53:53 hloc INFO] Importing matches into the database...
 11%|█         | 4128/37950 [00:06<00:42, 790.69it/s]

In [ ]:
%%time
print("\nRunning COLMAP image undistorter...\n")

# Running COLMAP image undistorter
undistorted_path = location / "undistorted"
!rm -rf {undistorted_path}

!colmap image_undistorter \
	--image_path /notebooks/working/input \
	--input_path {sfm_dir} \
	--output_path {undistorted_path} \
	--output_type COLMAP \
	# --max_image_size 2000

!mv {undistorted_path}/sparse {undistorted_path}/sparse_tmp
!mkdir {undistorted_path}/sparse
!mv {undistorted_path}/sparse_tmp {undistorted_path}/sparse/0


Running COLMAP image undistorter...


Reading reconstruction

 => Reconstruction with 13 images and 8 points

Image undistortion

Undistorting image [1/13]
Undistorting image [2/13]
Undistorting image [3/13]
Undistorting image [4/13]
Undistorting image [5/13]
Undistorting image [6/13]
Undistorting image [7/13]
Undistorting image [8/13]
Undistorting image [9/13]
Undistorting image [10/13]
Undistorting image [11/13]
Undistorting image [12/13]
Undistorting image [13/13]
Writing reconstruction...
Writing configuration...
Writing scripts...
Elapsed time: 0.022 [minutes]
CPU times: user 24.3 ms, sys: 342 ms, total: 366 ms
Wall time: 4.84 s


In [ ]:
!python /notebooks/gaussian-splatting/train.py -s {undistorted_path} -r 2


Optimizing 
Output folder: ./output/0754245b-1 [15/10 00:24:48]
Reading camera 13/13 [15/10 00:24:50]
Converting point3d.bin to .ply, will happen only the first time you open the scene. [15/10 00:24:50]
Loading Training Cameras [15/10 00:24:50]
Loading Test Cameras [15/10 00:24:54]
Number of points at initialisation :  8 [15/10 00:24:54]
Training progress:  23%|▋  | 7000/30000 [01:41<10:05, 38.02it/s, Loss=0.0552295]
[ITER 7000] Evaluating train: L1 0.02671748260036111 PSNR 27.612894439697268 [15/10 00:26:36]

[ITER 7000] Saving Gaussians [15/10 00:26:36]
Training progress:  64%|█▎| 19200/30000 [09:59<08:04, 22.30it/s, Loss=0.0076302]